### Import required classes

In [2]:
import numpy as np
import os
os.chdir('..')
from src.graph.semantic_network import SemanticNetwork
from src.text.text_wrangler import Corpus
from gensim.models import Word2Vec
from src.algorithms.hill_climbing import HillClimber
from src.algorithms import irt

### Create Network

In [3]:
shakespeare = Corpus("data/input/shakespeare.txt")
model = Word2Vec(shakespeare.sentence_matrix, size = 120,
                 window = 5, min_count=5, workers=8, sg=1)
network = SemanticNetwork(embeddings=model.wv.vectors, aligned_keys=model.wv.index2word)

### Training for 10 iterations, can increase if desired

In [4]:
for i in range(10):
    model.train(shakespeare.sentence_matrix, total_examples=len(shakespeare.sentence_matrix),
                epochs=1, compute_loss=True)
    print("Round {} ==================".format(i))
    network.update(em_proportion=1, g_proportion=1, include_set=shakespeare.nouns, stop_set=shakespeare.stopwords, thresh=0.8, verbose=True)

Round 0 ==================
Updated 4165828 edges
Round 1 ==================
Updated 2403888 edges
Round 2 ==================
Updated 1196578 edges
Round 3 ==================
Updated 525020 edges
Round 4 ==================
Updated 226694 edges
Round 5 ==================
Updated 105464 edges
Round 6 ==================
Updated 56390 edges
Round 7 ==================
Updated 35178 edges
Round 8 ==================
Updated 24512 edges
Round 9 ==================
Updated 18918 edges


### Begin Random Start Hill Climbing

* If no "start" specified, will randomize. 
* Allows consecutive "repeats" up to 3 times, then jumps to another random node.
    * Use "repeat" = # to specify in constructor
* Max out after 20 iterations. Use num_iter = # to specify in constructor.

In [17]:
walk = HillClimber(network.graph)
walk.run()

['deceas',
 'fornication',
 'fornication',
 'compliment',
 'compliment',
 'wantonness',
 'wantonness',
 'repetition',
 'repetition',
 'wantonness',
 'wantonness',
 'repetition',
 'repetition',
 'wantonness',
 'indignation',
 'rent',
 'rent',
 'carcass',
 'carcass',
 'goat',
 'goat',
 'carcass',
 'carcass',
 'goat',
 'goat',
 'carcass',
 'chariot',
 'smock',
 'smock',
 'sweetness',
 'sweetness',
 'station',
 'station',
 'sweetness',
 'sweetness',
 'station',
 'station',
 'sweetness',
 'brine',
 'scythe',
 'scythe']

### Calculate IRT

In [18]:
irt.IRT.calculate(walk.list)

[(None, 'deceas', 0),
 ('deceas', 'fornication', 0),
 ('fornication', 'compliment', 1),
 ('compliment', 'wantonness', 1),
 ('wantonness', 'repetition', 1),
 ('repetition', 'indignation', 6),
 ('indignation', 'rent', 0),
 ('rent', 'carcass', 1),
 ('carcass', 'goat', 1),
 ('goat', 'chariot', 6),
 ('chariot', 'smock', 0),
 ('smock', 'sweetness', 1),
 ('sweetness', 'station', 1),
 ('station', 'brine', 6),
 ('brine', 'scythe', 0)]

### Hill Climbing with defined start

In [6]:
walk = HillClimber(network.graph, start = "trifle")
walk.run()

['trifle',
 'lesson',
 'lesson',
 'wantonness',
 'wantonness',
 'repetition',
 'repetition',
 'wantonness',
 'wantonness',
 'repetition',
 'repetition',
 'wantonness',
 'stone',
 'sand',
 'sand',
 'trod',
 'trod',
 'hoop',
 'hoop',
 'goat',
 'goat',
 'carcass',
 'carcass',
 'goat',
 'goat',
 'carcass',
 'carcass',
 'goat',
 'conclusion',
 'path',
 'path',
 'transform',
 'transform',
 'path',
 'path',
 'transform',
 'transform',
 'path',
 'oil',
 'corruption',
 'corruption']

### Calculate IRT

In [14]:
irt.IRT.calculate(walk.list)

[(None, 'trifle', 0),
 ('trifle', 'lesson', 0),
 ('lesson', 'wantonness', 1),
 ('wantonness', 'repetition', 1),
 ('repetition', 'stone', 6),
 ('stone', 'sand', 0),
 ('sand', 'trod', 1),
 ('trod', 'hoop', 1),
 ('hoop', 'goat', 1),
 ('goat', 'carcass', 1),
 ('carcass', 'conclusion', 6),
 ('conclusion', 'path', 0),
 ('path', 'transform', 1),
 ('transform', 'oil', 6),
 ('oil', 'corruption', 0)]